In [1]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
im = Image.open('chad/1985.jpg')


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import os
sat_images = {}
years = []

for filename in os.listdir('chad/'):
    if filename.endswith(".jpg"):
        sat_images[filename] = Image.open('chad/' + filename)
        years.append(filename[:-4])

In [1]:
#sat_images

In [2]:
def get_water(sat_images):
    yearly_lake_size = []
    
    for year in sat_images:
        lake_size = 0
        im_array = np.array(sat_images[year])
    
        sh = im_array.shape
        y_cent = int(sh[0] / 2)
        x_cent = int(sh[1] / 2)
               
        ref_pix1 = im_array[y_cent][x_cent]
    
        ref_pix2 = im_array[y_cent + 100][x_cent + 100]
    
        ref_pix3 = im_array[y_cent + 100][x_cent - 100]

    
        refs = [ref_pix1, ref_pix2, ref_pix3]

        min_r = 255
        min_g = 255
        min_b = 255
    
        max_r = 0
        max_g = 0
        max_b = 0
    
        for ref in refs:
            if ref[0] < min_r:
                min_r = ref[0] - 10
        
            if ref[1] < min_g:
                min_g = ref[1] - 10
        
            if ref[2] < min_b:
                min_b = ref[2] - 10
            
            if ref[0] > max_r:
                max_r = ref[0] + 10
        
            if ref[1] > max_g:
                max_g = ref[1] + 10
        
            if ref[2] > max_b:
                max_b = ref[2] + 10
            
        for row in im_array:
            for pixel in row:
                if pixel[2] <= max_b and pixel[2] >= min_b and pixel[0] >= min_r and pixel[0] <= max_r and pixel[1] >= min_g and pixel[1] <= max_g:
                    lake_size += 1
                    pixel[0] = 0
                    pixel[1] = 0
                    pixel[2] = 245
                
        yearly_lake_size.append(lake_size)
        
        im = Image.fromarray(im_array)
        im.save(year + "processed.jpg")

In [6]:
year = "2000.jpg"
im_array = np.array(sat_images[year])
print(im_array.shape)

(3251, 4800, 3)


In [ ]:
yearly_lake_size = get_water(sat_images)

In [19]:
df = pd.DataFrame(years[:-1], columns = ['year'])
df['water_pixels'] = np.array(yearly_water_pix)
df = df.sort_values(by=['year'])

In [ ]:
plt.figure(figsize=(20, 8))

plt.plot(df['year'], df['water_pixels'])
plt.ylabel("Size of lake")
plt.show()